In [55]:
##ESTA CELDA PERMITE JUGAR A FLAPPY BIRD
#Pulsa cualquier tecla para saltar

import pygame
from pygame.locals import *  # noqa
import sys
import random


class FlappyBird_Human:
    def __init__(self):
        self.screen = pygame.display.set_mode((400, 700))
        self.bird = pygame.Rect(65, 50, 50, 50)
        self.background = pygame.image.load("assets/background.png").convert()
        self.birdSprites = [pygame.image.load("assets/1.png").convert_alpha(),
                            pygame.image.load("assets/2.png").convert_alpha(),
                            pygame.image.load("assets/dead.png")]
        self.wallUp = pygame.image.load("assets/bottom.png").convert_alpha()
        self.wallDown = pygame.image.load("assets/top.png").convert_alpha()
        self.gap = 145
        self.wallx = 400
        self.birdY = 350
        self.jump = 0
        self.jumpSpeed = 15
        self.gravity = 10
        self.dead = False
        self.sprite = 0
        self.counter = 0
        self.offset = random.randint(-200, 200)

    def updateWalls(self):
        self.wallx -= 4
        if self.wallx < -80:
            self.wallx = 400
            self.counter += 1
            self.offset = random.randint(-200, 200)

    def birdUpdate(self):
        if self.jump:
            self.jumpSpeed -= 1
            self.birdY -= self.jumpSpeed
            self.jump -= 1
        else:
            self.birdY += self.gravity
            self.gravity += 0.2
        self.bird[1] = self.birdY
        upRect = pygame.Rect(self.wallx,
                             360 + self.gap - self.offset + 10,
                             self.wallUp.get_width() - 10,
                             self.wallUp.get_height())
        downRect = pygame.Rect(self.wallx,
                               0 - self.gap - self.offset - 10,
                               self.wallDown.get_width() - 10,
                               self.wallDown.get_height())
        if upRect.colliderect(self.bird):
            self.dead = True
        if downRect.colliderect(self.bird):
            self.dead = True
        if not 0 < self.bird[1] < 720:
            self.bird[1] = 50
            self.birdY = 50
            self.dead = False
            self.counter = 0
            self.wallx = 400
            self.offset = random.randint(-110, 110)
            self.gravity = 10

    def run(self):
        clock = pygame.time.Clock()
        pygame.font.init()
        font = pygame.font.SysFont("Arial", 50)
        while True:
            
            clock.tick(60)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    sys.exit()
                if (event.type == pygame.KEYDOWN or event.type == pygame.MOUSEBUTTONDOWN) and not self.dead:
                    self.jump = 17
                    self.gravity = 10
                    self.jumpSpeed = 15

            self.screen.fill((255, 255, 255))
            self.screen.blit(self.background, (0, 0))
            self.screen.blit(self.wallUp,
                             (self.wallx, 360 + self.gap - self.offset))
            self.screen.blit(self.wallDown,
                             (self.wallx, 0 - self.gap - self.offset))
            self.screen.blit(font.render(str(self.counter),
                                         -1,
                                         (255, 255, 255)),
                             (200, 50))
            if self.dead:
                self.sprite = 2
            elif self.jump:
                self.sprite = 1
            self.screen.blit(self.birdSprites[self.sprite], (70, self.birdY))
            if not self.dead:
                self.sprite = 0
            self.updateWalls()
            self.birdUpdate()
            pygame.display.update()

if __name__ == "__main__":
    FlappyBird_Human().run()

KeyboardInterrupt: 

In [50]:
#Esta celda es una modificación de la anterior
#Se ha eliminado toda la parte gráfica y el juego no se ejecuta infinitamente
#La función run se ha sustituido por la función tick() que ejecuta un paso del juego cada vez que se le llama
#Se ha añadido también la función que devuelve la entrada de la red neuronal
#=> distancia X al tubo, distancia Y al tubo superior, distancia Y al tubo inferior, distancia al techo, distancia al suelo
#y una función que devuelve la distancia a la que ha llegado al pájaro y otra que devuelve si está vivo o no

import pygame
from pygame.locals import *  # noqa
import sys
import random


class FlappyBird:
    
def __init__(self):
       
        self.bird = pygame.Rect(65, 50, 50, 50)
        self.distance = 0
        self.gap = 145
        self.wallx = 400
        self.birdY = 350
        self.jump = 0
        self.jumpSpeed = 15
        self.gravity = 10
        self.dead = False
        self.counter = 0
        self.offset = random.randint(-200, 200)
        
    def calculateInput(self):
        dist_X_to_The_Wall = self.wallx+80
        dist_Y_to_The_Wall_UP = self.birdY-(0 - self.gap - self.offset+500)
        dist_Y_to_The_Wall_DOWN = self.birdY-(360 + self.gap - self.offset)
        dist_Y_TOP = self.birdY
        dist_Y_BOTTOM = 720-self.birdY
        res = [dist_X_to_The_Wall,dist_Y_to_The_Wall_UP,dist_Y_to_The_Wall_DOWN,dist_Y_TOP,dist_Y_BOTTOM]
        return res
    
    def isDead(self):
        return self.dead
    
    def TotalDistance(self):
        return self.distance


    
    def updateWalls(self):
        self.wallx -= 4
        if self.wallx < -80:
            self.wallx = 400
            self.counter += 1
            self.offset = random.randint(-200, 200)

    def birdUpdate(self):
        self.distance =  self.distance + 1 
        if self.jump:
            self.jumpSpeed -= 1
            self.birdY -= self.jumpSpeed
            self.jump -= 1
        else:
            self.birdY += self.gravity
            self.gravity += 0.2
        self.bird[1] = self.birdY
        upRect = pygame.Rect(self.wallx,
                             360 + self.gap - self.offset + 10,
                             88,
                             500)
        downRect = pygame.Rect(self.wallx,
                               0 - self.gap - self.offset - 10,
                               88,
                               500)
        if upRect.colliderect(self.bird):
            self.dead = True
        if downRect.colliderect(self.bird):
            self.dead = True
        if not 0 < self.bird[1] < 720:
            self.dead=True
            
        

    def tick(self,jump):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit()
            if (jump==True) and not self.dead:
                self.jump = 17
                self.gravity = 10
                self.jumpSpeed = 15
                
        self.updateWalls()
        self.birdUpdate()
        pygame.display.update()

        

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 30)

In [51]:
#Igual que la anterior, pero con la parte gráfica

class FlappyBird_GAME:
    


    def __init__(self):
        self.screen = pygame.display.set_mode((400, 700))
        self.bird = pygame.Rect(65, 50, 50, 50)
        self.background = pygame.image.load("assets/background.png").convert()
        self.birdSprites = [pygame.image.load("assets/1.png").convert_alpha(),
                            pygame.image.load("assets/2.png").convert_alpha(),
                            pygame.image.load("assets/dead.png")]
        self.wallUp = pygame.image.load("assets/bottom.png").convert_alpha()
        self.wallDown = pygame.image.load("assets/top.png").convert_alpha()
        self.distance = 0
        self.gap = 145
        self.wallx = 400
        self.birdY = 350
        self.jump = 0
        self.jumpSpeed = 15
        self.gravity = 10
        self.dead = False
        self.counter = 0
        self.offset = random.randint(-200, 200)
        self.sprite = 0
        
    def calculateInput(self):
        dist_X_to_The_Wall = self.wallx+80
        dist_Y_to_The_Wall_UP = self.birdY-(0 - self.gap - self.offset+500)
        dist_Y_to_The_Wall_DOWN = self.birdY-(360 + self.gap - self.offset)
        dist_Y_TOP = self.birdY
        dist_Y_BOTTOM = 720-self.birdY
        res = [dist_X_to_The_Wall,dist_Y_to_The_Wall_UP,dist_Y_to_The_Wall_DOWN,dist_Y_TOP,dist_Y_BOTTOM]
        return res
    
    def isDead(self):
        return self.dead
    
    def TotalDistance(self):
        return self.distance


    
    def updateWalls(self):
        self.wallx -= 4
        if self.wallx < -80:
            self.wallx = 400
            self.counter += 1
            self.offset = random.randint(-200, 200)

    def birdUpdate(self):
        self.distance =  self.distance + 1 
        if self.jump:
            self.jumpSpeed -= 1
            self.birdY -= self.jumpSpeed
            self.jump -= 1
        else:
            self.birdY += self.gravity
            self.gravity += 0.2
        self.bird[1] = self.birdY
        upRect = pygame.Rect(self.wallx,
                             360 + self.gap - self.offset + 10,
                             88,
                             500)
        downRect = pygame.Rect(self.wallx,
                               0 - self.gap - self.offset - 10,
                               88,
                               500)
        if upRect.colliderect(self.bird):
            self.dead = True
        if downRect.colliderect(self.bird):
            self.dead = True
        if not 0 < self.bird[1] < 720:
            self.dead=True
            
        

    def tick(self,jump):
        if (jump==True) and not self.dead:
                self.jump = 17
                self.gravity = 10
                self.jumpSpeed = 15
    

        self.screen.fill((255, 255, 255))
        self.screen.blit(self.background, (0, 0))
        self.screen.blit(self.wallUp,
                             (self.wallx, 360 + self.gap - self.offset))
    
        self.screen.blit(self.wallDown,
                             (self.wallx, 0 - self.gap - self.offset))
        self.screen.blit(font.render(str(self.counter),
                                         -1,
                                         (255, 255, 255)),
                             (200, 50))
        if self.dead:
            self.sprite = 2
        elif self.jump:
            self.sprite = 1
        self.screen.blit(self.birdSprites[self.sprite], (70, self.birdY))
        if not self.dead:
            self.sprite = 0
        self.updateWalls()
        self.birdUpdate()
        pygame.display.update()
        
       

        

In [52]:
#Neuroevolución con NEAT 
#El fitness es la distancia a la que ha llegado cada pájaro

import neat

number_generations = 1000
def eval_genomes(genomes,config):
    for genome_id, genome in genomes:
        genome.fitness = 99999
        net = neat.nn.FeedForwardNetwork.create(genome,config)
        bird = FlappyBird()
        while (not bird.allDead() and not bird.maxTotalDistance()>110000):
            nnInput = bird.calculateInput()
            #print(nnInput)
            #print(bird.fitness())
            output = net.activate(nnInput)
            if output[0] > output[1]:
                bird.tick(True)
            else:
                bird.tick(False)
            
        genome.fitness = bird.TotalDistance()
        
       
        

config = neat.Config(neat.DefaultGenome,neat.DefaultReproduction,neat.DefaultSpeciesSet,neat.DefaultStagnation,'FlapyBirdNEAT')

p = neat.Population(config)

p.add_reporter(neat.StdOutReporter(False))

winner = p.run(eval_genomes,number_generations)




 ****** Running generation 0 ****** 

Population's average fitness: 34.48000 stdev: 15.04691
Best fitness: 72.00000 - size: (2, 10) - species 1 - id 4
Average adjusted fitness: 0.202
Mean genetic distance 1.183, standard deviation 0.339
Population of 25 members in 1 species
Total extinctions: 0
Generation time: 0.020 sec

 ****** Running generation 1 ****** 

Population's average fitness: 50.20000 stdev: 22.56369
Best fitness: 86.00000 - size: (2, 10) - species 1 - id 29
Average adjusted fitness: 0.413
Mean genetic distance 1.135, standard deviation 0.349
Population of 25 members in 1 species
Total extinctions: 0
Generation time: 0.027 sec (0.024 average)

 ****** Running generation 2 ****** 

Population's average fitness: 62.40000 stdev: 20.53874
Best fitness: 87.00000 - size: (2, 10) - species 1 - id 29
Average adjusted fitness: 0.603
Mean genetic distance 1.087, standard deviation 0.261
Population of 25 members in 1 species
Total extinctions: 0
Generation time: 0.026 sec (0.025 ave

Average adjusted fitness: 0.174
Mean genetic distance 1.646, standard deviation 0.593
Population of 25 members in 2 species
Total extinctions: 0
Generation time: 0.038 sec (0.030 average)

 ****** Running generation 32 ****** 

Population's average fitness: 97.24000 stdev: 73.48974
Best fitness: 343.00000 - size: (2, 6) - species 2 - id 567
Average adjusted fitness: 0.228
Mean genetic distance 1.488, standard deviation 0.703
Population of 25 members in 2 species
Total extinctions: 0
Generation time: 0.033 sec (0.031 average)

 ****** Running generation 33 ****** 

Population's average fitness: 91.24000 stdev: 49.01125
Best fitness: 204.00000 - size: (2, 5) - species 2 - id 770
Average adjusted fitness: 0.351
Mean genetic distance 1.750, standard deviation 0.614
Population of 25 members in 2 species
Total extinctions: 0
Generation time: 0.031 sec (0.032 average)

 ****** Running generation 34 ****** 

Population's average fitness: 63.08000 stdev: 24.36870
Best fitness: 103.00000 - size:

In [71]:
#Esta celda visualiza el mejor genoma tras la neuroevolución

clock = pygame.time.Clock()
pygame.font.init()
font = pygame.font.SysFont("Arial", 50)
bird = FlappyBird_GAME()

while (not bird.isDead()):
    clock.tick(60)
    net = neat.nn.FeedForwardNetwork.create(winner,config)
    nnInput = bird.calculateInput()
    output = net.activate(nnInput)
    if output[0] > output[1]:
        bird.tick(True)
    else:
        bird.tick(False)
        
print(bird.TotalDistance())
        

KeyboardInterrupt: 